## Purpose
**This notebook is a work in progress.** This notebook explores natural keys for a quarter of contract data.

In [2]:
# from pudl_rmi.process_eqr import engine, DATE_COLUMNS

import pandas as pd
import sqlalchemy as sa

ModuleNotFoundError: No module named 'sqlalchemy'

## EQR Sales

In [2]:
engine = sa.create_engine("sqlite:////Users/bendnorman/catalyst/rmi-ferc1-eia/outputs/eqr.db")

with engine.connect() as conn:
    contracts = pd.read_sql("select * from contracts", conn)
    identities = pd.read_sql("select * from identities", conn)

In [3]:
contracts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778269 entries, 0 to 778268
Data columns (total 33 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   contract_unique_id                     778269 non-null  object 
 1   seller_company_name                    778269 non-null  object 
 2   seller_history_name                    0 non-null       object 
 3   customer_company_name                  778269 non-null  object 
 4   contract_affiliate                     778269 non-null  object 
 5   ferc_tariff_reference                  778241 non-null  object 
 6   contract_service_agreement_id          778189 non-null  object 
 7   contract_execution_date                778269 non-null  object 
 8   commencement_date_of_contract_term     778269 non-null  object 
 9   contract_termination_date              260131 non-null  object 
 10  actual_termination_date                15911 non-null   

In [4]:
identities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 16 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   filer_unique_id                                  22998 non-null  object
 1   company_name                                     22998 non-null  object
 2   company_identifier                               22998 non-null  object
 3   contact_name                                     22998 non-null  object
 4   contact_title                                    22998 non-null  object
 5   contact_address                                  22998 non-null  object
 6   contact_city                                     22998 non-null  object
 7   contact_state                                    22998 non-null  object
 8   contact_zip                                      22998 non-null  object
 9   contact_country_name                   

In [5]:
identities = identities.query("quarter == 'Q1'")
contracts = contracts.query("quarter == 'Q1'")

## Find contracts natural key

In [35]:
# nk_fields = ["seller_company_name", "contract_service_agreement_id", "ferc_tariff_reference", "customer_company_name"]
nk_fields = ["seller_company_name", "customer_company_name", "ferc_tariff_reference", "contract_service_agreement_id", "product_name", "quantity", "rate"]
contracts[nk_fields].isna().any()

seller_company_name              False
customer_company_name            False
ferc_tariff_reference             True
contract_service_agreement_id     True
product_name                     False
quantity                          True
rate                              True
dtype: bool

In [36]:
duplicate_contracts = contracts.duplicated(subset=nk_fields)
duplicate_contracts.value_counts()

False    130775
True      46755
dtype: int64

In [38]:
contracts[duplicate_contracts]

,contract_unique_id,seller_company_name,seller_history_name,customer_company_name,contract_affiliate,ferc_tariff_reference,contract_service_agreement_id,contract_execution_date,commencement_date_of_contract_term,contract_termination_date,...,rate_description,rate_units,point_of_receipt_balancing_authority,point_of_receipt_specific_location,point_of_delivery_balancing_authority,point_of_delivery_specific_location,begin_date,end_date,year,quarter
64,C1,Cadillac Renewable Energy LLC,None,Consumers Energy Company,N,FERC Electric Rate Schedule No. 1 Consumers PPA,Consumers PPA,1997-05-09 00:00:00.000000,1997-05-09 00:00:00.000000,2028-07-16 00:00:00.000000,...,See Amendment No. 5 to PPA at Section 8(b)(B) ...,None,None,None,CONS,CONS.CADILLGMGA2,None,None,2020,Q1
66,C1,Cadillac Renewable Energy LLC,None,Consumers Energy Company,N,FERC Electric Rate Schedule No. 1 Consumers PPA,Consumers PPA,1997-05-09 00:00:00.000000,1997-05-09 00:00:00.000000,2028-07-16 00:00:00.000000,...,See Amendment No. 5 to PPA at Section 8(b)(A) ...,None,None,None,CONS,CONS.CADILLGMGA2,None,None,2020,Q1
67,C1,Cadillac Renewable Energy LLC,None,Consumers Energy Company,N,FERC Electric Rate Schedule No. 1 Consumers PPA,Consumers PPA,1997-05-09 00:00:00.000000,1997-05-09 00:00:00.000000,2028-07-16 00:00:00.000000,...,See Amendment No. 5 to PPA at Section 8(a)(i) ...,None,None,None,CONS,CONS.CADILLGMGA2,None,None,2020,Q1
68,C1,Cadillac Renewable Energy LLC,None,Consumers Energy Company,N,FERC Electric Rate Schedule No. 1 Consumers PPA,Consumers PPA,1997-05-09 00:00:00.000000,1997-05-09 00:00:00.000000,2028-07-16 00:00:00.000000,...,See Amendment No. 5 to PPA at Section 8(a)(ii)...,None,None,None,CONS,CONS.CADILLGMGA2,None,None,2020,Q1
70,C1,Cadillac Renewable Energy LLC,None,Consumers Energy Company,N,FERC Electric Rate Schedule No. 1 Consumers PPA,Consumers PPA,1997-05-09 00:00:00.000000,1997-05-09 00:00:00.000000,2028-07-16 00:00:00.000000,...,See Amendment No. 5 to PPA at Section 8(c)(B) ...,None,None,None,CONS,CONS.CADILLGMGA2,None,None,2020,Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177523,C11,"Upper Missouri G. & T. Electric Cooperative, Inc.",None,"Southeast Electric Cooperative, Inc.",N,FERC Electric Tariff Vol. No. 1,SA11,1972-03-29 00:00:00.000000,2015-10-01 00:00:00.000000,2075-12-31 00:00:00.000000,...,FERC Electric Tariff Vol. No. 1 filed in Docke...,None,None,None,None,None,None,None,2020,Q1
177526,C11,"Upper Missouri G. & T. Electric Cooperative, Inc.",None,"Southeast Electric Cooperative, Inc.",N,FERC Electric Tariff Vol. No. 1,SA11,1972-03-29 00:00:00.000000,2015-10-01 00:00:00.000000,2075-12-31 00:00:00.000000,...,FERC Electric Tariff Vol. No. 1 filed in Docke...,None,None,None,None,None,None,None,2020,Q1
177527,C11,"Upper Missouri G. & T. Electric Cooperative, Inc.",None,"Southeast Electric Cooperative, Inc.",N,FERC Electric Tariff Vol. No. 1,SA11,1972-03-29 00:00:00.000000,2015-10-01 00:00:00.000000,2075-12-31 00:00:00.000000,...,FERC Electric Tariff Vol. No. 1 filed in Docke...,None,None,None,None,None,None,None,2020,Q1
177528,C11,"Upper Missouri G. & T. Electric Cooperative, Inc.",None,"Southeast Electric Cooperative, Inc.",N,FERC Electric Tariff Vol. No. 1,SA11,1972-03-29 00:00:00.000000,2015-10-01 00:00:00.000000,2075-12-31 00:00:00.000000,...,FERC Electric Tariff Vol. No. 1 filed in Docke...,None,None,None,None,None,None,None,2020,Q1


In [ ]:
contracts.set_index()

In [24]:
contracts.contract_unique_id.value_counts().describe()

count    20079.000000
mean         8.841576
std        101.556610
min          1.000000
25%          1.000000
50%          2.000000
75%          5.000000
max      10288.000000
Name: contract_unique_id, dtype: float64

In [18]:
contracts.contract_service_agreement_id.isna().value_counts()

False    177510
True         20
Name: contract_service_agreement_id, dtype: int64

In [6]:
contracts.columns

Index(['contract_unique_id', 'seller_company_name', 'seller_history_name',
       'customer_company_name', 'contract_affiliate', 'ferc_tariff_reference',
       'contract_service_agreement_id', 'contract_execution_date',
       'commencement_date_of_contract_term', 'contract_termination_date',
       'actual_termination_date', 'extension_provision_description',
       'class_name', 'term_name', 'increment_name', 'increment_peaking_name',
       'product_type_name', 'product_name', 'quantity', 'units', 'rate',
       'rate_minimum', 'rate_maximum', 'rate_description', 'rate_units',
       'point_of_receipt_balancing_authority',
       'point_of_receipt_specific_location',
       'point_of_delivery_balancing_authority',
       'point_of_delivery_specific_location', 'begin_date', 'end_date', 'year',
       'quarter'],
      dtype='object')

In [15]:
identities.company_identifier.value_counts().describe()

count    2759.000000
mean        2.035883
std         1.066914
min         1.000000
25%         2.000000
50%         2.000000
75%         2.000000
max        53.000000
Name: company_identifier, dtype: float64

In [24]:
seller_identities = identities[identities.filer_unique_id.str.contains("FS")]
seller_identities.duplicated(["filer_unique_id", "company_identifier"]).value_counts()

False    2859
dtype: int64

In [26]:
seller_identities.filer_unique_id.value_counts()

FS1    2753
FS2     103
FS3       3
Name: filer_unique_id, dtype: int64

In [27]:
seller_identities.query("filer_unique_id == 'FS3'")

,filer_unique_id,company_name,company_identifier,contact_name,contact_title,contact_address,contact_city,contact_state,contact_zip,contact_country_name,contact_phone,contact_email,transactions_reported_to_index_price_publishers,filing_quarter,year,quarter
111,FS3,"Avalon Solar Partners, LLC",C004572,Che Geiser,"Director, Asset Management","150 E Colorado Blvd, Ste 100",Pasadena,CA,91105,US,530-747-2026,assetoperations@coronalenergy.com,N,202003,2020,Q1
422,FS3,"Ridge Crest Wind Partners, LLC",C002062,Matthew Moody,Assoc. Asset Manager,15445 Innovation Drive,San Diego,CA,92128,US,858-521-2928,matthew.moody@edf-re.com,N,202003,2020,Q1
2808,FS3,"Merrill Lynch Commodities, Inc.",C001801,LaKeisha Rowe,AVP,20 East Greenway Plaza,Houston,TX,77046,US,8326815978,lakeisha.d.rowe@bofa.com,N,202003,2020,Q1


In [32]:
seller_identities.groupby("company_identifier").nunique().filer_unique_id.sort_values()

company_identifier
C000029    1
C004746    1
C004756    1
C004757    1
C004758    1
          ..
C002354    2
C003329    2
C001801    3
C004572    3
C002062    3
Name: filer_unique_id, Length: 2753, dtype: int64

In [34]:
identities.query("company_identifier == 'C001801'")

,filer_unique_id,company_name,company_identifier,contact_name,contact_title,contact_address,contact_city,contact_state,contact_zip,contact_country_name,contact_phone,contact_email,transactions_reported_to_index_price_publishers,filing_quarter,year,quarter
2804,FS1,"Merrill Lynch Commodities, Inc.",C001801,Suzette Guerrero,Vice President,20 East Greenway Plaza,Houston,TX,77046,US,8326815699,suzette.guerrero@baml.com,N,202003,2020,Q1
2805,FA1,"Merrill Lynch Commodities, Inc.",C001801,Suzette Guerrero,Vice President,20 East Greenway Plaza,Houston,TX,77046,US,8326815699,suzette.guerrero@baml.com,N,202003,2020,Q1
2806,FS2,"Merrill Lynch Commodities, Inc.",C001801,Lindsay Thomas,AVP,20 East Greenway Plaza,Houston,TX,77046,US,8326815333,lthomas6@bofa.com,N,202003,2020,Q1
2807,FA2,"Merrill Lynch Commodities, Inc.",C001801,Lindsay Thomas,AVP,20 East Greenway Plaza,Houston,TX,77046,US,8326815333,lthomas6@bofa.com,N,202003,2020,Q1
2808,FS3,"Merrill Lynch Commodities, Inc.",C001801,LaKeisha Rowe,AVP,20 East Greenway Plaza,Houston,TX,77046,US,8326815978,lakeisha.d.rowe@bofa.com,N,202003,2020,Q1
2809,FA3,"Merrill Lynch Commodities, Inc.",C001801,LaKeisha Rowe,AVP,20 East Greenway Plaza,Houston,TX,77046,US,8326815978,lakeisha.d.rowe@bofa.com,N,202003,2020,Q1


In [39]:
duplicate_ids = identities.duplicated(["filer_unique_id", "company_identifier"], keep=False)
duplicate_ids.value_counts()

False    5530
True       87
dtype: int64

In [43]:
identities[duplicate_ids].drop_duplicates().shape

(16, 16)

In [51]:
identities[duplicate_ids].shape

(87, 16)

In [49]:
identities[duplicate_ids].drop_duplicates().sort_values(by="company_identifier")

,filer_unique_id,company_name,company_identifier,contact_name,contact_title,contact_address,contact_city,contact_state,contact_zip,contact_country_name,contact_phone,contact_email,transactions_reported_to_index_price_publishers,filing_quarter,year,quarter
3649,FA1,Macquarie Energy LLC,C001161,Kevin Sabouni,Senior Associate,500 Dallas Street\r\nSuite 3300,Houston,TX,77002,US,713-275-8883,Kevin.Sabouni@macquarie.com,N,202003,2020,Q1
5546,FA1,Macquarie Energy LLC,C001161,Kevin Sabouni,Executive,500 Dallas Street\r\nSuite 3300,Houston,TX,77002,US,713-275-8883,Kevin.Sabouni@macquarie.com,Y,202003,2020,Q1
2411,FA1,Venus Tech Inc,D003630,Vengaiah Mutthineni,CEO,13568 Cedar Run Ln,Herndon,VA,20171,US,202-683-6263,etariff@venustechinc.com,N,202003,2020,Q1
124,FA1,Energy New England,D003636,George Korbas,Senior Energy Analyst,100 Foxborough Blvd.,Foxborough,MA,02035,US,508-698-1220,gkorbas@energynewengland.com,N,202003,2020,Q1
883,FA1,Energy New England,D003636,George Korbas,Senior Energy Analyst,5 Hampshire Street,Mansfield,MA,02048,US,508-698-1220,gkorbas@energynewengland.com,N,202003,2020,Q1
2819,FA1,Alliance For Cooperative Energy Services Power...,D003672,Scott Petrin,Manager of Bilateral Settlements,4140 West 99th Street,Carmel,IN,46032,US,317-344-7208,scottp@acespower.com,N,202003,2020,Q1
3138,FA1,Alliance For Cooperative Energy Services Power...,D003672,Renee McDonald,Lead Settlements Analyst,4140 West 99th Street,Carmel,IN,46032,US,317-344-7082,reneem@acespower.com,N,202003,2020,Q1
4628,FA1,Alliance For Cooperative Energy Services Power...,D003672,Joseph Edenborough,Settlements Analyst - Bilateral,4140 West 99th Street,Carmel,IN,46032,US,317-344-7005,jedenborough@acespower.com,N,202003,2020,Q1
4880,FA1,Alliance For Cooperative Energy Services Power...,D003672,Ross Pierson,Settlements Analyst - Bilateral,4140 W. 99th St.,Carmel,IN,46032,US,317-344-7078,rpierson@acespower.com,N,202003,2020,Q1
5383,FA1,Alliance For Cooperative Energy Services Power...,D003672,Scott Petrin,Manager Bilateral Settlements,4140 W. 99th St.,Carmel,IN,46032,US,317-344-7208,scottp@acespower.com,N,202003,2020,Q1


In theory, comapny_identifier and filer_unique_id should create a natural key. Most do but there are 87 records with duplicate natural keys. Most are true duplicates. There are 16 records that have the same natural keys but different contact information. I think this is due to a failure to increment the integer after FA and FS. 

Potential clean algorithm: for a given quarter, drop all pure duplicates, correct filer_unique_id integers for records with duplicate natural keys (filer_unique_id, company_identifier)

## Entergy

In [69]:
identities["company_name"] = identities["company_name"].str.lower()

companies = identities.drop_duplicates(subset=["company_identifier", "company_name"])
assert companies.company_identifier.is_unique
assert companies.company_name.is_unique

AssertionError: 

In [70]:
companies[companies.company_identifier.duplicated(keep=False)]

,filer_unique_id,company_name,company_identifier,contact_name,contact_title,contact_address,contact_city,contact_state,contact_zip,contact_country_name,contact_phone,contact_email,transactions_reported_to_index_price_publishers,filing_quarter,year,quarter
5169,FS1,emera maine,C000120,Briana Littlefield,Sales and Revenue Analyst,PO Box 932,Bangor,ME,04401,US,207-973-2808,briana.littlefield@versantpower.com,N,202003,2020,Q1
5170,FA1,versant power,C000120,Karen Bell,Contract Administrator,PO Box 932,Bangor,ME,04401,US,207-973-2821,karen.bell@versantpower.com,N,202003,2020,Q1
5214,FA1,"west medway ii, llc",C005271,Rosemary Pistel,"Manager, Trade Compliance",1310 Point Street 8th Floor'',Baltimore,MD,21231,US,410-470-3185,Rosemary.Pistel@constellation.com,N,202003,2020,Q1
5215,FS1,"exelon west medway ii, llc",C005271,Marianne Alvarez,Sr Analyst Federal Reg Affairs,101 Constitution Ave. NW,Washington,DC,20001,US,202-637-0358,Marianne.Alvarez@exeloncorp.com,N,202003,2020,Q1
5288,FS1,"plymouth rock energy, llc",C002647,Kenneth Lackey,Vice President - Merchant Operations,"1990 Post Oak Blvd., Suite 1900",Houston,TX,77056,US,(713) 636-1992,Ken.Lackey@engie.com,N,202003,2020,Q1
5290,FA1,engie power & gas llc,C002647,Lieu Vo,Risk and Compliance,"1360 Post Oak Blvd, ste. 400",Houston,TX,77056,US,(713) 636-1374,Lieu.Vo@engie.com,N,202003,2020,Q1


Oy vey, there are a small number of companies with the same company_identifier but with different names. Instead of extracting the company name and id from the identities table, maybe we use the main FERC CID dataset (assuming it has unique ids!)

I'm going to drop the the duplicates for now.

In [71]:
companies = companies.drop_duplicates(subset=["company_identifier"])

### Search using company_name

In [72]:
entergy_ids = identities[identities.company_name.str.lower().str.contains("entergy")]
print(entergy_ids.shape)
entergy_ids.drop_duplicates(subset=["company_identifier"]).company_name.value_counts()

(22, 16)


entergy nuclear indian point 2, llc     1
entergy nuclear indian point 3, llc     1
entergy services, llc                   1
entergy power, llc                      1
entergy nuclear palisades, llc          1
entergy nuclear power marketing, llc    1
entergy arkansas, llc                   1
entergy louisiana, llc                  1
entergy mississippi, llc                1
entergy new orleans, llc                1
entergy texas, inc.                     1
Name: company_name, dtype: int64

### Search using email addresses

In [73]:
entergy_ids = identities[identities.contact_email.str.lower().str.contains("entergy")]
print(entergy_ids.shape)
entergy_ids.drop_duplicates(subset=["company_identifier"]).company_name.value_counts()

(30, 16)


entergy nuclear indian point 2, llc     1
entergy nuclear indian point 3, llc     1
rs cogen, llc                           1
eam nelson holding, llc                 1
system energy resources, inc.           1
ewo marketing, llc                      1
entergy services, llc                   1
entergy power, llc                      1
entergy nuclear palisades, llc          1
entergy nuclear power marketing, llc    1
entergy arkansas, llc                   1
entergy louisiana, llc                  1
entergy mississippi, llc                1
entergy new orleans, llc                1
entergy texas, inc.                     1
Name: company_name, dtype: int64

Looks like 